In [1]:
import numpy as np
import pandas as pd
from NeuralNetwork import *
from utils import *
import warnings
# to ignore pandas warning
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
%%time
grid={
    "eta" : [0.001,0.0001,0.00001,0.01,0.1],
    "mb" : [1,2,4,8,16,32,64,128,256,700],
    "momentum" : [0.9,0.8,0.95,0.97,0.99],
    "n_layers" : [1],
    "n_neurons" : [200,225,250, 400],
    "epochs" : [500],
    "clip_value" : [None],
    "hid_act_fun" : ["relu"],
    "out_act_fun" : ["linear"],
    "cost_fun" : ["eucl"],
    "ridge_lambda": [None],
    "lasso_lambda": [None],
    "decay_max_steps": [None],
    "decay_min_value": [None],
    "es_patience": [50]
} 

col_names = ('Input1', 'Input2', 'Input3', 'Input4', 'Input5', 'Input6',
             'Input7', 'Input8', 'Input9', 'Input10', 'TARGET_x', 'TARGET_y', 'TARGET_z')
data = pd.read_csv("../ML-23-PRJ-Package/ML-CUP23-TR.csv",
                   skiprows=7, usecols=[i for i in range(1, 14)], names=col_names)

data = data.sample(frac=1)
#SPLIT 80/20 FOR TRAIN/TEST
folds = np.array_split(data,10 )
tr_data = pd.concat(folds[2:8])
test_data = pd.concat(folds[0:1])
es_data=folds[9]

search_space=get_search_space(grid)
print(len(search_space))
parallel_grid_search(5, tr_data,es_data, search_space, 10, 3,type="cup",verbose="yes",refined=True)

1000
N_cores = 10


KeyboardInterrupt: 

EARLY STOPPED
